### Features för analys 
Skapa nya kolumner för analys (t.ex. veckodag, månad, försäljning per vecka, ledtider)

- Utgår fran passadatum - Därför att de beskriver när aktivitet faktiskt sker.

**Tid (när passet sker)**
- **veckodag**: vilken dag, t.ex. måndag.
- **vecka**: vilken vecka på året.
- **månad**: vilken månad.
- **år**: vilket år.
- **helg**: är det lördag/söndag? (ja/nej)
- **tid_på_dag**: natt, morgon, dag eller kväll.

**Bokning**
- **ledtid_dagar**: hur många dagar innan passet man bokade.

**Efterfrågan (hur populärt det är)**
- **pass_popularitet_vecka**: hur många som bokat samma pass den veckan.
- **anläggning_belastning_vecka**: hur många bokningar gymmet haft den veckan.

**Ny medlemstyp (utifrån pris)**
- **medlemstyp_ny**: gruppen baserat på vad man betalar.
  - 0 kr = **VIP**
  - 249 kr = **Student**
  - 349 kr = **Student+** eller **Bas** (beroende på vad personen var innan)
  - 449 kr = **Student+** eller **Bas+**
  - 599 kr = **Premium**

In [13]:
# Importera bibliotek
import sqlite3
import pandas as pd
import numpy as np

In [ ]:
# Läs in all data från SQLite
conn = sqlite3.connect('friskvard_data_cleaned.db')
df_all = pd.read_sql('SELECT * FROM friskvard_data', conn)
conn.close

<function Connection.close()>

In [18]:
# Skapa en kopia för feature engineering
df_feat = df_all.copy()
df_feat

,bokning_id,medlem_id,medlemstyp,medlem_startdatum,medlem_slutdatum,månadskostnad,födelseår,pass_id,passnamn,anläggning,...,bokningsdatum,passdatum,passtid,status,feedback_text,feedbackdatum,feedback_betyg,är_negativt_belopp,månadskostnad_abs,dataset
0,BOK-000001,MED-10158,Premium,2023-07-29 00:00:00,2025-03-07 00:00:00,599,2006.0,PASS-2024-10-01-001,Yoga,Göteborg Centrum,...,2024-09-24 00:00:00,2024-01-10 00:00:00,11:00:00.000000,Genomförd,Bästa yogapasset jag varit på. Kommer tillbaka!,2024-02-10 00:00:00,4.0,0,599,main
1,BOK-000002,MED-10229,Premium,2023-01-08 00:00:00,2025-06-26 00:00:00,599,1984.0,PASS-2024-12-04-002,Spinning,Malmö Centrum,...,2024-11-27 00:00:00,2024-04-12 00:00:00,10:00:00.000000,Genomförd,Professionellt och välorganiserat pass.,2024-05-12 00:00:00,5.0,0,599,main
2,BOK-000003,MED-10223,Student,2023-12-27 00:00:00,2025-12-05 00:00:00,249,2006.0,PASS-2024-11-09-003,Styrketräning,Okänd,...,2024-08-11 00:00:00,2024-09-11 00:00:00,18:00:00.000000,Genomförd,,NaN,NaN,0,249,main
3,BOK-000004,MED-10110,Bas,2023-07-03 00:00:00,2025-03-24 00:00:00,349,2006.0,PASS-2024-09-04-004,Pilates,Göteborg Centrum,...,2024-01-09 00:00:00,2024-09-04 00:00:00,07:00:00.000000,Genomförd,Professionellt och välorganiserat pass.,2024-02-09 00:00:00,5.0,0,349,main
4,BOK-000005,MED-10022,Premium,2022-05-11 00:00:00,2025-01-06 00:00:00,599,1992.0,PASS-2024-10-29-005,Pilates,Västerås,...,2024-10-26 00:00:00,2024-10-29 00:00:00,20:30:00.000000,Genomförd,Bästa pilatespasset jag varit på. Kommer tillb...,2024-10-30 00:00:00,5.0,0,599,main
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,BOK-002996,MED-10037,Premium,2024-03-13 00:00:00,2024-12-10 00:00:00,599,2003.0,PASS-2024-10-24-2996,Boxning,Stockholm Kungsholmen,...,2024-10-19 00:00:00,2024-10-24 00:00:00,17:30:00.000000,Genomförd,,NaN,NaN,0,599,main
2996,BOK-002997,MED-10199,Bas,2022-07-25 00:00:00,2024-09-12 00:00:00,349,2004.0,PASS-2024-12-03-2997,Styrketräning,Örebro,...,2024-11-30 00:00:00,2024-03-12 00:00:00,18:30:00.000000,No-Show,,NaN,NaN,0,349,main
2997,BOK-002998,MED-10073,Premium,2023-02-19 00:00:00,2024-08-26 00:00:00,-349,1982.0,PASS-2024-10-03-2998,Boxning,Malmö Centrum,...,2024-09-26 00:00:00,2024-03-10 00:00:00,19:00:00.000000,Genomförd,,NaN,NaN,1,349,main
2998,BOK-002999,MED-10151,Bas,2023-10-11 00:00:00,2024-09-09 00:00:00,349,1996.0,PASS-2024-07-03-2999,Boxning,Malmö Centrum,...,2024-06-29 00:00:00,2024-03-07 00:00:00,07:30:00.000000,Genomförd,Instruktören var superbra på att korrigera tek...,2024-03-07 00:00:00,4.0,0,349,main


In [ ]:
df_feat['anläggning'].len()

anläggning
Malmö Västra Hamnen      272
Stockholm Södermalm      256
Uppsala                  254
Stockholm City           251
Göteborg Hisingen        249
Västerås                 246
Göteborg Centrum         243
Lund                     238
Malmö Centrum            234
Stockholm Kungsholmen    228
Linköping                222
Örebro                   212
Okänd                     95
Name: count, dtype: int64

In [ ]:
# === Tidsegenskaper för passdatum ===
df_feat['veckodag'] = df_feat['passdatum'].dt.day_name()
df_feat['vecka'] = df_feat['passdatum'].dt.isocalendar().week.astype('Int64')
df_feat['månad'] = df_feat['passdatum'].dt.month.astype('Int64')
df_feat['år'] = df_feat['passdatum'].dt.year.astype('Int64')

# === Region ===
def map_region(df):
    region_map = {
    'Stockholmn City': 'Syd',
    'Stockholm Södermalm': 'Syd',
    'Stockholm Kungsholmen': 'Syd',
    'Uppsala': 'Syd',
    'Västerås': 'Syd',
    'Linköping': 'Syd',
    'Örebro': 'Syd',
    'Göteborg Centrum': 'Väst',
    'Göreborg Hisingen': 'Väst',
    'Malmö Centrum': 'Syd',
    'Malmö Västra Hamnen': 'Syd',
    'Lund': 'Syd',
    'Okänd': 'Övrigt'
}
    df_feat = df.copy()    
    df_feat['region'] = df_feat['anläggning'].map(region_map)
    return df_feat
df_feat = map_region(df_feat)

### Sentimentalanalys
Klassificera recensioner/feedback som positiv, neutral eller negativ med BERT eller LLMlanalys

In [ ]:
# device=0 använder GPU, device=-1 använder CPU
# Importera bibliotek
from transformers import pipeline

# Ladda sentimentmodellen
classifier = pipeline(
    "sentiment-analysis",
    model="KBLab/robust-swedish-sentiment-multiclass",
    device=0 if torch.cuda.is_available() else -1
)

In [ ]:
# === Enkel sentimentanalys ===

# Små listor med positiva/negativa svenska ord
pos_ord = {
    'bra','bäst','fantastisk','toppen','nöjd','nöjda','älskar','älskade','trevlig','trevliga',
    'snäll','snälla','hjälpsam','hjälpsamma','proffsig','proffsiga','ren','fräsch','fräscht',
    'kul','roligt','rekommenderar','gillar','glad','super','klockren','perfekt'
}
neg_ord = {
    'dålig','dåligt','sämst','missnöjd','tråkig','tråkigt','besviken','dyr','smutsig','stökig',
    'otrevlig','otrevligt','kall','varm','fel','problem','jobbigt','kaos','aldrig','inte','försenad'
}

def enkel_sentiment(text):
    if pd.isna(text) or not str(text).strip():
        return 'neutral'
    text = str(text).lower()
    pos = sum(ord in text for ord in pos_ord)
    neg = sum(ord in text for ord in neg_ord)
    if pos > neg:
        return 'positiv'
    if neg > pos:
        return 'negativ'
    return 'neutral'

df_feat['sentiment_enkel'] = df_feat['feedback_text'].apply(enkel_sentiment).astype('category')

df_feat['sentiment_enkel'].value_counts(dropna=False)